In [ ]:
#SEE IF graphs look better with or without "%matplotlib inline" for you;
from __future__ import print_function
%matplotlib inline
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage.filters import gaussian_filter

from collections import namedtuple
from mpl_toolkits.mplot3d import Axes3D

bench_dir = 'bench_py_v011'

size_ag, size_lex, size_mem = 32, 32, 32

#mrspan_start, mrspan_stop, mrspan_points = 0.0, 2.0, 20+1
mrspan_start, mrspan_stop, mrspan_points = 0.05, 1.00, 19+1

mr_seq = np.linspace(mrspan_start, mrspan_stop, num=mrspan_points, endpoint = True)
plt.rcParams['figure.figsize'] = (15,15)
plt.rcParams['figure.dpi'] = 150

data = dict()

for fname in sorted(os.listdir(bench_dir)):
    match = re.search("{([0-9\.]+)}_{([A-Za-z].+)}", fname)
    if match:
        mr_val = float(match.group(1))
        mutator = match.group(2)
        if mutator not in data:
            data[mutator] = dict()

        ffname = os.path.join(bench_dir, fname)
        df = pd.read_csv(ffname, comment='#', sep=' ')
        if not len(df):
            print("NO DATA! - %s" % fname)
        else:
            if df.MeanGrammarDrift[1] == 0:
                print("NO DRIFT - %s"%fname)
            else:
                df['NormMeanGrammarDrift'] = df.MeanGrammarDrift/df.MeanGrammarDrift[1]
                data[mutator][mr_val] = df
                print("LOADED   - %s" % fname)
            
for mutator, dfs in data.items():
    max_comprehension = max([0,]+[df.Comprehension.max() for df in dfs.values()])
    if max_comprehension < 0.7:
        print("Mutator %s never reached comprehension above 0.7, tossing out" % mutator)
        del data[mutator]

In [ ]:
#PLOT TIME SERIES
#PLOT Mutation min, average, max across runs values, by mutator (can also look at percentiles, just in case)
varlims = {'Comprehension':[0,1],'NormMeanGrammarDrift': [0,2], 'MeanGrammarDrift': [0,0.02], 'AgentGrammarDrift': [0,0.02]}

run_indexes = {}
for mutator in data.keys():
    run_indexes[mutator] = {}
    ckeys = data[mutator].keys()
    vals = np.array([ data[mutator][k].Comprehension for k in ckeys ])
    finalvals = vals[:,-1]
    for ndx, f in enumerate(finalvals):
        if f == np.percentile(finalvals, 25, interpolation='nearest'):
            run_indexes[mutator]['low'] = ckeys[ndx]
        if f == np.percentile(finalvals, 50, interpolation='nearest'):
            run_indexes[mutator]['med'] = ckeys[ndx]
        if f == np.percentile(finalvals, 100, interpolation='nearest'):
            run_indexes[mutator]['high'] = ckeys[ndx]

for varexam in ['Comprehension','NormMeanGrammarDrift']:
    obsnum = 100

    multirun_med = dict()
    multirun_low = dict()
    multirun_high = dict()


    #subplots columns, 4 seems the most readable
    spCols = 4
    
    f, unfl_axarr = plt.subplots(int((len(data)-1)/spCols)+1, spCols, figsize=(13,9))
    axarr=unfl_axarr.flatten()

    for axindex, (mutator, rundata) in enumerate(data.items()):
        plt.subplot(axarr[axindex])
        y      = rundata[run_indexes[mutator]['med' ]][varexam]
        y_low  = rundata[run_indexes[mutator]['low' ]][varexam]
        y_high = rundata[run_indexes[mutator]['high']][varexam]

        plt.title("%s\n%s" % (varexam, mutator))

        plt.plot(y_high, label = 'hi comp')
        plt.plot(y, label = 'med comp')
        plt.plot(y_low, label = 'lo comp')
        plt.fill_between(range(len(y_low)), y_low, y_high, alpha = .2, color = 'grey' )
        
        #plt.semilogy()
        
        if varexam in varlims:
            plt.ylim(varlims[varexam])

        if (axindex == 0):
            plt.legend(loc = 'upper right')
        
        # print("Min/Max %s values for mutator %s are %.3f, %.3f" % (varexam, mutator, min(y), max(y)) )

    plt.tight_layout()
    plt.show()


In [ ]:
for mutator, mutatordata in data.items():
    fig = plt.figure(figsize=(12,7))

    for varndx, varexam in enumerate(['Comprehension','NormMeanGrammarDrift']):
        ax = fig.add_subplot(2,2,1+varndx, projection='3d')

        x = sorted(mutatordata.keys())
        y = range(50, obsnum)

        X, Y = np.meshgrid(x, y)

        zs = np.array( [mutatordata[j][varexam][k] for k in y for j in x] )
        Z = zs.reshape(X.shape)

        ax.plot_surface(X, Y, Z, cmap = 'RdBu_r', alpha = 0.8)

        ax.set_xticklabels(x)
        ax.set_title(mutator)
        ax.set_xlabel('Mutation Scale')
        ax.set_ylabel('Iteration')
        ax.set_zlabel(varexam)

        if varexam in varlims:
            ax.set_zlim(varlims[varexam])

    plt.tight_layout()

    plt.show()
    


In [ ]:
from matplotlib.pyplot import cm 

colors = []
mutator_ixs = {}
color=cm.jet(np.linspace(0,1,len(data)))

plt.figure(figsize=(5,5))
for mndx, (mutator, mutatordata) in enumerate(data.items()):
    if mutator not in mutator_ixs:
        mutator_ixs[mutator] = len(mutator_ixs)
    final_comprehension = []
    final_drift = []
    for mr_val, df in mutatordata.items():
        final_comprehension.append(df.iloc[-1].Comprehension)
        final_drift.append(df.iloc[-1].NormMeanGrammarDrift)
    plt.scatter(final_comprehension, final_drift, s=40, lw=0, c=color[mndx], label=mutator)
#plt.ylim([0.1,2])
plt.semilogy()
plt.xlim([0.5,1])
plt.ylabel('NormMeanGrammarDrift')
plt.xlabel('Comprehension')
plt.legend(bbox_to_anchor=(2.1,1.0))

In [ ]:
for k in data['ProbitVectorUniform']:
    print(k, data['ProbitVectorUniform'][k].MeanGrammarDrift)